# An intro to using Python in Jupyter notebooks for data science
## Common tasks and grammar of data manipulation

## Overview

**Overall purpose**: The purpose of these repository notebooks is to perform a classification task on the selected dataset to classify whether, according to some predictors, a person's income will exceed $50k.  The dataset includes several predictors, but the ones that we will use for classification are `hours-per-week` (how many hours are worked per week), `workclass` (broad category of the type of work performed by the person), and `age` (how old the person is, in years). 

**Notebook purpose**: This notebook will illustrate some data manipulation tasks, and show how the grammar of data manipulation \[select, filter, mutate, summarise, arrange\] is achieved through Pandas and Python.  Keep in mind that there are *many* ways to achieve a certain grammar operation, and the syntax in this notebook absolutely does not demonstrate the exhaustive set.  

**Data**: In this notebook, we demonstrate a data science task on "cleaned data" using Pandas, a popular tool in Python for analyzing data.  Since the data we will use is rather small, it is included in the repository under the data folder.  The included dataset is a slightly modified version of the Census income dataset obtained from the [UC Irvine machine learning data repository](https://archive.ics.uci.edu/ml/datasets/adult).

**Cleaning tasks**:  This dataset has leading spaces after the delimiter in the .CSV file, as well as some non-standard indications of missing values (`?`).  For simplication, all the rows with any NA are simply dropped.

The cleaned data will be saved to the local data directory, and then used in the following notebook, which will show some common machine learning tasks using scikit-learn.

Similarly to R, we start our processing by importing the packages that we need.  We will use pandas for our processing, which provides functionality for manipulating tabular data and other data types.

In [ ]:
#import statements
import pandas as pd

In [ ]:
#magics
%matplotlib inline

## Loading and viewing the data

In [ ]:
na_list = []
na_list = ['?']

In [ ]:
#Load data
filename = 'data/adult_data.csv'
df = pd.read_csv(filename, skipinitialspace=True, na_values=na_list)

In [ ]:
# Get a preview of the data
df.head()

In [ ]:
df.info(null_counts=True) #But what does pandas consider to be null/na?

In [ ]:
df.info(verbose=False)

In [ ]:
df.describe()

In [ ]:
#Count na values
df.isnull().sum(skipna=False)

In [ ]:
df.columns

In [ ]:
# Look at the value counts for all of the columns: categorical histogram
for x in df.columns:
    if df[x].dtype=='object':
        print('Col name: ', x, '\n', df[x].value_counts(), '\n')

## Handling missing values

In [ ]:
#Simplest approach if datset large enough: drop the missing values
#Note: avoid performing operations on subsets of a data frame
test = df.dropna(axis=0)
print('# of rows of returned (test) data frame: ', len(test))
print('# of rows of original (df) data frame: ', len(df))

In [ ]:
df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
print('# of rows of original (df) data frame: ', len(df))

In [ ]:
df.isnull().sum(skipna=False)

In [ ]:
#Can write out data frames as well as read them!
out_filename = 'data/adult_clean.csv'
df.to_csv(out_filename, index=False)

## Select: choose data from the dataframe

In [ ]:
#Select certain columns by name and/or index
#Note: avoid performing operations on subsets of a data frame
occupation_info = df['occupation'] #shorthand
occupation_info = df.loc[:, 'occupation'] #using loc operator
occupation_info = df.loc[:, ['occupation', 'marital-status', 'relationship']] #select multiple columns using a list
occupation_info = df.iloc[:, [5,6,7]] #select columns using their integer location

In [ ]:
occupation_info.head()

In [ ]:
#Select certain rows
first_n_rows = df.loc[0:10, :] #select rows by index NAME
first_n_rows = df.iloc[0:10, :] #select rows by integer location

Try it yourself!  Fill in the code that would select the desired data for each comment.

In [ ]:
#Select the 'capital-gain' column  by name.  Can it you also do it using the index? (hint: iloc)

In [ ]:
#Select the first 5 rows.

## Filter: choose rows of data based on some criteria

In [ ]:
#Select columns according to some criteria
nm_bools = df.loc[:, 'marital-status'] != 'Never-married'
n_nm_df = df.loc[nm_bools, :]
n_nm_df.head()

In [ ]:
#Select and view data for the 'Private' workclass
n_np_df = df.loc[ df.loc[:, 'workclass'] == 'Private']
n_np_df.head()

## Mutate: Perform operations on columns to create another

In [ ]:
#Just selection and indexing to simply create another column in the df
df.loc[:, 'age_adj'] = df.loc[:, 'age'] + 1
#df['age_adj'] = df['age'] + 1
df.head()

In [ ]:
#Use assign for a more R-like feel; columns must have keyword syntax
df = df.assign(signed_capital = df['capital-gain'] - df['capital-loss'])
df.rename(columns={'signed_capital' : 'signed-capital'})
df.head()

In [ ]:
#Perform pipeline-like behavior using previous computations in the computation for future computations
df = df.assign(hours_per_year = df['hours-per-week']*52,
               hours_per_life = lambda y: y['hours_per_year'] * df['age'])
df.rename(columns={ 'hours_per_year': 'hours-per-year',
                   'hours_per_life': 'hours-per-life'}, inplace=True)

In [ ]:
df.head()

## Groupby and summarize: group items by some desired similarity

In [ ]:
#Calculate mean capital-gain based on workclass
workclass_count = df.groupby(['workclass']).count()
workclass_count.head()

In [ ]:
#Calculate mean capital-gain based on workclass and marital status
work_rel_mean = df.groupby(['workclass', 'marital-status']).mean()
work_rel_mean

In [ ]:
df.describe()

In [ ]:
df.sum(axis=0, numeric_only=True)

## Arrange: change the ordering of the data

In [ ]:
#Sort data according to age
df.sort_values(by='age')

In [ ]:
df.sort_values(by='age', ascending=False)

In [ ]:
df.sort_values(by=['age', 'education-num'])

In [ ]:
df.sort_values(by=['age', 'education-num'], ascending=[False, True])

## Plotting examples

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
??pd.Series.hist

In [ ]:
#You can plot directly using Pandas!  Pandas plots are based on the matplotlib library.
df['education-num'].hist(bins=15);
plt.title('Histogram of education-nums')
plt.xlabel('Education-num')
plt.ylabel('Frequency (counts)')

In [ ]:
workclass_pie_info = df.groupby('workclass').count().mode(axis=1)

plt.figure(figsize=(10,6))
plt.pie(x=workclass_pie_info.iloc[:,0], labels = workclass_pie_info.index);